### Importing libraries

In [148]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
!pip install xgboost
!pip install category_encoders

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 72.4/72.4 kB 189.5 kB/s eta 0:00:00


## Import data

In [149]:
Ames = pd.read_csv('AmesHousing.csv')
Train = pd.read_csv('train.csv')
Test = pd.read_csv('test.csv')

In [150]:
Ames.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,1960,1960,Hip,CompShg,BrkFace,Plywood,Stone,112.0,TA,TA,CBlock,TA,Gd,Gd,BLQ,639.0,Unf,0.0,441.0,1080.0,GasA,Fa,Y,SBrkr,1656,0,0,1656,1.0,0.0,1,0,3,1,TA,7,Typ,2,Gd,Attchd,1960.0,Fin,2.0,528.0,TA,TA,P,210,62,0,0,0,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,7,5,1968,1968,Hip,CompShg,BrkFace,BrkFace,None,0.0,Gd,TA,CBlock,TA,TA,No,ALQ,1065.0,Unf,0.0,1045.0,2110.0,GasA,Ex,Y,SBrkr,2110,0,0,2110,1.0,0.0,2,1,3,1,Ex,8,Typ,2,TA,Attchd,1968.0,Fin,2.0,522.0,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [151]:
Train.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706,Unf,0,150,856,GasA,Ex,Y,SBrkr,856,854,0,1710,1,0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2,548,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978,Unf,0,284,1262,GasA,Ex,Y,SBrkr,1262,0,0,1262,0,1,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2,460,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486,Unf,0,434,920,GasA,Ex,Y,SBrkr,920,866,0,1786,1,0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2,608,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216,Unf,0,540,756,GasA,Gd,Y,SBrkr,961,756,0,1717,1,0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3,642,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655,Unf,0,490,1145,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1,0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3,836,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


## Check features names (to make sure there is no inconsistency)

In [152]:
Test.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1461,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal
1,1462,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal
2,1463,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal
3,1464,60,RL,78.0,9978,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,6,1998,1998,Gable,CompShg,VinylSd,VinylSd,BrkFace,20.0,TA,TA,PConc,TA,TA,No,GLQ,602.0,Unf,0.0,324.0,926.0,GasA,Ex,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,Gd,7,Typ,1,Gd,Attchd,1998.0,Fin,2.0,470.0,TA,TA,Y,360,36,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal
4,1465,120,RL,43.0,5005,Pave,NaN,IR1,HLS,AllPub,Inside,Gtl,StoneBr,Norm,Norm,TwnhsE,1Story,8,5,1992,1992,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,263.0,Unf,0.0,1017.0,1280.0,GasA,Ex,Y,SBrkr,1280,0,0,1280,0.0,0.0,2,0,2,1,Gd,5,Typ,0,NaN,Attchd,1992.0,RFn,2.0,506.0,TA,TA,Y,0,82,0,0,144,0,NaN,NaN,NaN,0,1,2010,WD,Normal


In [153]:
cols_dic = dict(Ames_cols= Ames.columns,
               Train_cols = Train.columns,
               Test_cols= Test.columns)
cols_df = pd.DataFrame.from_dict(cols_dic, orient='index').transpose()
cols_df


,Ames_cols,Train_cols,Test_cols
0,Order,Id,Id
1,PID,MSSubClass,MSSubClass
2,MS SubClass,MSZoning,MSZoning
3,MS Zoning,LotFrontage,LotFrontage
4,Lot Frontage,LotArea,LotArea
5,Lot Area,Street,Street
6,Street,Alley,Alley
7,Alley,LotShape,LotShape
8,Lot Shape,LandContour,LandContour
9,Land Contour,Utilities,Utilities


In [154]:
Ames = Ames.drop(['Order', 'PID'], axis = 1)
Train = Train.drop('Id', axis = 1)

In [155]:
cols_dic = dict(zip(Ames.columns, Train.columns))


In [156]:
Ames = Ames.rename(columns=cols_dic)

In [157]:
df_train = pd.concat([Train, Ames])


## Drop duplications and resetting index

In [158]:
df_train = df_train.drop_duplicates()
df_train = df_train.reset_index().drop('index', axis = 1)
df_train['Id'] = range(len(df_train))
df_train

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,Id
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2003,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,196.0,Gd,TA,PConc,Gd,TA,No,GLQ,706.0,Unf,0.0,150.0,856.0,GasA,Ex,Y,SBrkr,856,854,0,1710,1.0,0.0,2,1,3,1,Gd,8,Typ,0,NaN,Attchd,2003.0,RFn,2.0,548.0,TA,TA,Y,0,61,0,0,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500,0
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,6,8,1976,1976,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,CBlock,Gd,TA,Gd,ALQ,978.0,Unf,0.0,284.0,1262.0,GasA,Ex,Y,SBrkr,1262,0,0,1262,0.0,1.0,2,0,3,1,TA,6,Typ,1,TA,Attchd,1976.0,RFn,2.0,460.0,TA,TA,Y,298,0,0,0,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500,1
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2001,2002,Gable,CompShg,VinylSd,VinylSd,BrkFace,162.0,Gd,TA,PConc,Gd,TA,Mn,GLQ,486.0,Unf,0.0,434.0,920.0,GasA,Ex,Y,SBrkr,920,866,0,1786,1.0,0.0,2,1,3,1,Gd,6,Typ,1,TA,Attchd,2001.0,RFn,2.0,608.0,TA,TA,Y,0,42,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500,2
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,7,5,1915,1970,Gable,CompShg,Wd Sdng,Wd Shng,None,0.0,TA,TA,BrkTil,TA,Gd,No,ALQ,216.0,Unf,0.0,540.0,756.0,GasA,Gd,Y,SBrkr,961,756,0,1717,1.0,0.0,1,0,3,1,Gd,7,Typ,1,Gd,Detchd,1998.0,Unf,3.0,642.0,TA,TA,Y,0,35,272,0,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000,3
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,8,5,2000,2000,Gable,CompShg,VinylSd,VinylSd,BrkFace,350.0,Gd,TA,PConc,Gd,TA,Av,GLQ,655.0,Unf,0.0,490.0,1145.0,GasA,Ex,Y,SBrkr,1145,1053,0,2198,1.0,0.0,2,1,4,1,Gd,9,Typ,1,TA,Attchd,2000.0,RFn,3.0,836.0,TA,TA,Y,192,84,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4192,80,RL,37.0,7937,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,Mitchel,Norm,Norm,1Fam,SLvl,6,6,1984,1984,Gable,CompShg,HdBoard,HdBoard,None,0.0,TA,TA,CBlock,TA,TA,Av,GLQ,819.0,Unf,0.0,184.0,1003.0,GasA,TA,Y,SBrkr,1003,0,0,1003,1.0,0.0,1,0,3,1,TA,6,Typ,0,NaN,Detchd,1984.0,Unf,2.0,588.0,TA,TA,Y,120,0,0,0,0,0,NaN,GdPrv,NaN,0,3,2006,WD,Normal,142500,4192
4193,20,RL,NaN,8885,Pave,NaN,IR1,Low,AllPub,Inside,Mod,Mitchel,Norm,Norm,1Fam,1Story,5,5,1983,1983,Gable,CompShg,HdBoard,HdBoard,None,0.0,TA,TA,CBlock,Gd,TA,Av,BLQ,301.0,ALQ,324.0,239.0,864.0,GasA,TA,Y,SBrkr,902,0,0,902,1.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1983.0,Unf,2.0,484.0,TA,TA,Y,164,0,0,0,0,0,NaN,MnPrv,NaN,0,6,2006,WD,Normal,131000,4193
4194,85,RL,62.0,10441,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,SFoyer,5,5,1992,1992,Gable,CompShg,HdBoard,Wd Shng,None,0.0,TA,TA,PConc,Gd,TA,Av,GLQ,337.0,Unf,0.0,575.0,912.0,GasA,TA,Y,SBrkr,970,0,0,970,0.0,1.0,1,0,3,1,TA,6,Typ,0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,Y,80,32,0,0,0,0,NaN,MnPrv,Shed,700,7,2006,WD,Normal,132000,4194
4195,20,RL,77.0,10010,Pave,NaN,Reg,Lvl,AllPub,Inside,Mod,Mitchel,Norm,N

In [159]:
df_test = Test.copy()


## Target variable overview

#### Check if it is normally distributed

In [160]:
sns.displot(df_train['SalePrice'], kde = True)

In [161]:
df_train['SalePrice'] = df_train['SalePrice'].apply(np.log)

In [162]:
sns.displot(df_train['SalePrice'], kde = True)

In [163]:
all_data = pd.concat([df_train.drop('SalePrice', axis = 1), df_test])

## Missing data overview

Take in consideration the following points
In some datasets there is zero to imply this value is missing, especially in numerical features (instead of np.nan)
Missing data can be presented as the word "None" (instead of np.nan)

In [164]:
def miss_data(data):
    Miss_df = ((data == 0).sum()).to_frame()
    Miss_df = Miss_df.rename(columns = {0: 'zeros'})
    Miss_df.index.name = 'Feature'
    Miss_df['np.nan'] = (data.isnull()).sum()
    Miss_df['None'] = (data == "None").sum()
    Miss_df['total'] = Miss_df['zeros'] + Miss_df['np.nan'] + Miss_df['None']
    Miss_df['Percent'] = 100*Miss_df['total']/len(data)
    Miss_df['Type'] = [data[i].dtype for i in Miss_df.index ]
    return    Miss_df.sort_values(ascending = False, by = 'Percent')

In [165]:
### Missing data in all_data
miss_data(data = all_data)

,zeros,np.nan,None,total,Percent,Type
Feature,,,,,,
PoolQC,0,5634,0,5634,99.611033,object
PoolArea,5631,0,0,5631,99.557992,int64
3SsnPorch,5587,0,0,5587,98.780057,int64
LowQualFinSF,5577,0,0,5577,98.603253,int64
MiscVal,5454,0,0,5454,96.428571,int64
MiscFeature,0,5450,0,5450,96.357850,object
BsmtHalfBath,5309,4,0,5313,93.935644,float64
Alley,0,5272,0,5272,93.210750,object
ScreenPorch,5156,0,0,5156,91.159830,int64


By simple analysis of the previous dataframe we should do:
Mean imputation for: 'BsmtFullBath', 'BsmtHalfBath', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', 'GarageCars', 'GarageArea' (In df_train and df_test separately)
Most frequent imputation for: 'MasVnrType', 'GarageFinish', 'GarageQual', 'GarageCond', 'MSZoning', 'Utilities', 'Functional', 'Exterior2nd', 'Exterior1st', 'SaleType', 'KitchenQual' (In df_train and df_test separately)
Create new features to be used instead of the original ones that have a lot of zeros
'Porch' = 'OpenPorchSF' + 'EnclosedPorch' + '3SsnPorch' + 'ScreenPorch'
'Bath_total' = 'FullBath' + 0.5'HalfBath' + 'BsmtFullBath' + 0.5'BsmtHalfBath'
'BsmtFinSF' = 'BsmtFinSF1' + 'BsmtFinSF2' - 'BsmtUnfSF'
FinSF' = '1stFlrSF' + '2ndFlrSF' - 'LowQualFinSF'
Drop useless features:
'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch', 'FullBath', 'HalfBath' , 'BsmtFullBath' ,'BsmtHalfBath', 'BsmtFinSF1' , 'BsmtFinSF2' , 'BsmtUnfSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF'
Some features have high percentage of missing values (or zeros) that should be dropped
'PoolQC', 'PoolArea', 'MiscFeature', 'MiscVal', 'Alley', 'Fence', 'WoodDeckSF', 'FireplaceQu', 'Fireplaces', 'MasVnrType', 'MasVnrArea'
some features have np.nan values, but it sould be replaced by "None"
'BsmtQual','BsmtCond','BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'GarageType', 'GarageFinish', 'GarageQu', 'GarageCond'
Other zeros have reasons to be zero
np.nan values to be zeros:
GarageYrBlt should be zeros (but this feature sould be converted to categorical later)
GarageArea, GarageCars
np.nan values in 'MasVnrArea' should be zeros
LotFrontage' to be imputed using a regression model (In df_train and df_test separately)


In [166]:
cat_feat = ['GarageFinish', 'GarageQual', 'GarageCond', 'MSZoning',
                  'Utilities', 'Functional', 'Exterior2nd', 'Exterior1st', 'SaleType', 'KitchenQual']
num_feat = ['BsmtFullBath', 'BsmtHalfBath', 'GarageCars', 'GarageArea', 'TotalBsmtSF', 'BsmtUnfSF', 'BsmtFinSF2',
                  'BsmtFinSF1']

In [167]:
from sklearn.impute import SimpleImputer

In [168]:
cat_imputer_train = SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')
df_train.loc[:,cat_feat] = cat_imputer_train.fit_transform(df_train.loc[:, cat_feat])
num_imputer_train = SimpleImputer(missing_values= np.nan, strategy = 'mean')
df_train.loc[:, num_feat] = num_imputer_train.fit_transform(df_train.loc[:, num_feat])

In [169]:
cat_imputer_test = SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')
df_test.loc[:,cat_feat] = cat_imputer_test.fit_transform(df_test.loc[:, cat_feat])
num_imputer_test = SimpleImputer(missing_values= np.nan, strategy = 'mean')
df_test.loc[:, num_feat] = num_imputer_test.fit_transform(df_test.loc[:, num_feat])

In [170]:
df_train['Bath_total'] = df_train['FullBath'] + 0.5*df_train['HalfBath'] + df_train['BsmtFullBath'] + 0.5*df_train['BsmtHalfBath']
df_test['Bath_total'] = df_test['FullBath'] + 0.5*df_test['HalfBath'] + df_test['BsmtFullBath'] + 0.5*df_test['BsmtHalfBath']

In [171]:
df_train['BsmtFinSF'] = df_train['BsmtFinSF1'] + df_train['BsmtFinSF2'] - df_train['BsmtUnfSF']
df_test['BsmtFinSF'] = df_test['BsmtFinSF1'] + df_test['BsmtFinSF2'] - df_test['BsmtUnfSF']


In [172]:
df_train['Porch'] = df_train['OpenPorchSF'] + df_train['EnclosedPorch'] + df_train['3SsnPorch'] + df_train['ScreenPorch']
df_test['Porch'] = df_test['OpenPorchSF'] + df_test['EnclosedPorch'] + df_test['3SsnPorch'] + df_test['ScreenPorch']

In [173]:
df_train['FinSF'] = df_train['1stFlrSF'] + df_train['2ndFlrSF'] - df_train['LowQualFinSF']
df_test['FinSF'] = df_test['1stFlrSF'] + df_test['2ndFlrSF'] - df_test['LowQualFinSF']

In [174]:
cols_to_drop = ['PoolQC', 'PoolArea', 'MiscFeature', 'MiscVal', 'Alley', 'Fence', 'WoodDeckSF',
            'FireplaceQu', 'Fireplaces', 'MasVnrType', 'MasVnrArea',
            'FullBath', 'HalfBath' , 'BsmtFullBath' ,'BsmtHalfBath',
            'BsmtFinSF1' , 'BsmtFinSF2' , 'BsmtUnfSF', 'OpenPorchSF', 'EnclosedPorch', '3SsnPorch', 'ScreenPorch',
            '1stFlrSF', '2ndFlrSF', 'LowQualFinSF']

In [175]:
df_train = df_train.drop(cols_to_drop, axis = 1)
df_test = df_test.drop(cols_to_drop, axis = 1)


In [176]:
Cols_to_None = ['BsmtQual','BsmtCond','BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'GarageType', 
                        'GarageFinish', 'GarageQual', 'GarageCond']

In [177]:
df_train[Cols_to_None] = df_train.loc[:,Cols_to_None].replace(to_replace = np.nan, value = "None", inplace = False)
df_test[Cols_to_None] = df_test.loc[:,Cols_to_None].replace(to_replace = np.nan, value = "None", inplace = False)

In [178]:
cols_to_zero = ['GarageYrBlt', 'GarageArea', 'GarageCars']
df_train[cols_to_zero] = df_train[cols_to_zero].fillna(0)
df_test[cols_to_zero] = df_test[cols_to_zero].fillna(0)

In [179]:
df_train_totrain = df_train[df_train['LotFrontage'].isnull() == False]
df_train_totest = df_train[df_train['LotFrontage'].isnull() == True]
from xgboost import XGBRegressor
reg = XGBRegressor(n_estimators = 5000, learning_rate = 0.01)
reg.fit(df_train_totrain.select_dtypes(exclude = 'object').drop(['LotFrontage', 'SalePrice', 'Id'], axis = 1), 
        df_train_totrain['LotFrontage'], verbose = False)
df_train_totest['LotFrontage'] = reg.predict(df_train_totest.select_dtypes(exclude = 'object').drop(['LotFrontage', 'SalePrice', 'Id'], axis = 1))
df_train = pd.concat([df_train_totrain, df_train_totest])

In [180]:
df_test_totrain = df_test[df_test['LotFrontage'].isnull() == False]
df_test_totest = df_test[df_test['LotFrontage'].isnull() == True]
from xgboost import XGBRegressor
reg_test = XGBRegressor(n_estimators = 5000, learning_rate = 0.01)
reg_test.fit(df_test_totrain.select_dtypes(exclude = 'object').drop(['LotFrontage', 'Id'], axis = 1), df_test_totrain['LotFrontage'], 
        verbose = False)
df_test_totest['LotFrontage'] = reg_test.predict(df_test_totest.select_dtypes(exclude = 'object').drop(['LotFrontage', 'Id'], axis = 1))
df_test = pd.concat([df_test_totrain, df_test_totest])

In [181]:
df_train = df_train.sort_values('Id', ascending = True)
df_test = df_test.sort_values('Id', ascending = True)

In [182]:
### Missing data in traing dataset
print(miss_data(data = df_train))

               zeros  np.nan  None  total    Percent     Type
Feature                                                      
Porch           1295       0     0   1295  30.855373    int64
GarageYrBlt      232       0     0    232   5.527758  float64
GarageArea       230       0     0    230   5.480105  float64
GarageType         0       0   230    230   5.480105   object
GarageCars       230       0     0    230   5.480105  float64
BsmtFinSF        147       0     0    147   3.502502  float64
BsmtExposure       0       0   119    119   2.835359   object
BsmtFinType2       0       0   117    117   2.787706   object
BsmtFinType1       0       0   115    115   2.740052   object
BsmtCond           0       0   115    115   2.740052   object
BsmtQual           0       0   115    115   2.740052   object
TotalBsmtSF      114       0     0    114   2.716226  float64
BedroomAbvGr      14       0     0     14   0.333572    int64
KitchenAbvGr       4       0     0      4   0.095306    int64
Electric

In [183]:
### Missing data in testing dataset
miss_data(data = df_test)

,zeros,np.nan,None,total,Percent,Type
Feature,,,,,,
Porch,415,0,0,415,28.444140,int64
GarageYrBlt,78,0,0,78,5.346127,float64
GarageArea,76,0,0,76,5.209047,float64
GarageType,0,0,76,76,5.209047,object
GarageCars,76,0,0,76,5.209047,float64
BsmtFinSF,52,0,0,52,3.564085,float64
BsmtCond,0,0,45,45,3.084304,object
BsmtQual,0,0,44,44,3.015764,object
BsmtExposure,0,0,44,44,3.015764,object


### Dealing with categorical features

Date features should be converted to be categorical, but first:
  - There are many years present in df_train and not present in df_test and vice versa
  - To convert date features to categorical that will lead to a large number of categories. So, we will collect year features to be every five years as follows.
A categorical feature in train dataset may have unique categories that not in the test dataset or vice versa, so, we should compare unique categories first in both train and test datasets

In [184]:
date_cols = ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt']
df_train[date_cols] = round(df_train[date_cols]/5)*5
df_test[date_cols] = round(df_test[date_cols]/5)*5

## Check if there is any category in train dataset and not in test dataset or vice virsa

In [185]:
def compare_cols(cols):
    s=1
    for i in cols:
        print(f"feature is: {i}")
        train_unique_count = len(df_train[i].unique())
        test_unique_count = len(df_test[i].unique())
        print(f"# of train unique values is: {train_unique_count}")
        print(f"# of test unique values is: {test_unique_count}")
        all_values = set(df_train[i]).union(set(df_test[i]))
        D = pd.DataFrame({'Unique': list(all_values)})
        D['train_count'] = [len(df_train[df_train[i] == j]) for j in all_values]
        D['train_percent'] = (D['train_count']/ len(df_train)) * 100
        D['test_count'] = [len(df_test[df_test[i] == j]) for j in all_values]
        D['test_percent'] = (D['test_count']/ len(df_test)) * 100
        print(D.sort_values('train_count', ascending = False))
        print('--------------------')
        print(f"Total number of navigated features is: {s}")
        s+=1
        print('--------------------') 

In [186]:
compare_cols(cols = date_cols)

feature is: YearBuilt
# of train unique values is: 29
# of test unique values is: 26
    Unique  train_count  train_percent  test_count  test_percent
19  2005.0          759      18.084346         297     20.356408
17  2000.0          343       8.172504         111      7.607951
10  1970.0          291       6.933524          89      6.100069
8   1960.0          285       6.790565          97      6.648389
11  1975.0          268       6.385513          86      5.894448
9   1965.0          264       6.290207          89      6.100069
15  1995.0          258       6.147248          93      6.374229
7   1955.0          257       6.123421          89      6.100069
6   1950.0          171       4.074339          62      4.249486
12  1980.0          151       3.597808          53      3.632625
0   1920.0          147       3.502502          45      3.084304
4   1940.0          142       3.383369          51      3.495545
1   1925.0          136       3.240410          53      3.632625
14  1

In [187]:
df_train['YearBuilt'] = df_train['YearBuilt'].replace({1885.0:1890.0, 1870.0:1880.0, 1875.0:1880.0})
df_train['GarageYrBlt'] = df_train['GarageYrBlt'].replace(to_replace= 1905.0, value= 1910.0, inplace = False)
df_train['GarageYrBlt'] = df_train['GarageYrBlt'].replace(to_replace= 2205.0, value= 2005.0, inplace = False)
df_test['GarageYrBlt'] = df_test['GarageYrBlt'].replace(to_replace= 2205.0, value= 2005.0, inplace = False)

In [188]:
# Recheck that every thing is Ok
compare_cols(cols = date_cols)

feature is: YearBuilt
# of train unique values is: 26
# of test unique values is: 26
    Unique  train_count  train_percent  test_count  test_percent
17  2005.0          759      18.084346         297     20.356408
16  2000.0          343       8.172504         111      7.607951
10  1970.0          291       6.933524          89      6.100069
8   1960.0          285       6.790565          97      6.648389
11  1975.0          268       6.385513          86      5.894448
9   1965.0          264       6.290207          89      6.100069
15  1995.0          258       6.147248          93      6.374229
7   1955.0          257       6.123421          89      6.100069
6   1950.0          171       4.074339          62      4.249486
12  1980.0          151       3.597808          53      3.632625
0   1920.0          147       3.502502          45      3.084304
4   1940.0          142       3.383369          51      3.495545
1   1925.0          136       3.240410          53      3.632625
14  1

In [189]:
# Some features were found to be categorical to be more meaningful
df_train['MSSubClass'] = df_train['MSSubClass'].astype(str)#Convert 'MSSubClass' feature to categorical instead of numerical
df_train['YearBuilt'] = df_train['YearBuilt'].astype(str)
df_train['YearRemodAdd'] = df_train['YearRemodAdd'].astype(str)
df_train['GarageYrBlt'] = df_train['GarageYrBlt'].astype(str)
df_train['YrSold'] = df_train['YrSold'].astype(str)
df_train['MoSold'] = df_train['MoSold'].astype(str)

df_test['MSSubClass'] = df_test['MSSubClass'].astype(str)#Convert 'MSSubClass' feature to categorical instead of numerical
df_test['YearBuilt'] = df_test['YearBuilt'].astype(str)
df_test['YearRemodAdd'] = df_test['YearRemodAdd'].astype(str)
df_test['GarageYrBlt'] = df_test['GarageYrBlt'].astype(str)
df_test['YrSold'] = df_test['YrSold'].astype(str)
df_test['MoSold'] = df_test['MoSold'].astype(str)

In [190]:
# Review categorical features 
compare_cols(cols = df_train.select_dtypes(include= 'object').columns)

feature is: MSSubClass
# of train unique values is: 16
# of test unique values is: 16
   Unique  train_count  train_percent  test_count  test_percent
0      20         1524      36.311651         543     37.217272
3      60          828      19.728377         276     18.917066
7      50          422      10.054801         143      9.801234
6     120          259       6.171075          95      6.511309
15     30          204       4.860615          70      4.797807
12    160          186       4.431737          65      4.455106
9      70          185       4.407910          68      4.660727
10     80          174       4.145818          60      4.112406
5      90          156       3.716941          57      3.906785
14    190           87       2.072909          31      2.124743
11     85           67       1.596378          28      1.919123
13     75           38       0.905409           7      0.479781
1      45           30       0.714796           6      0.411241
4     180         

   Unique  train_count  train_percent  test_count  test_percent
4    1Fam         3488      83.106981        1205     82.590816
0  TwnhsE          324       7.719800         113      7.745031
2  Duplex          156       3.716941          57      3.906785
1   Twnhs          140       3.335716          53      3.632625
3  2fmCon           89       2.120562          31      2.124743
--------------------
Total number of navigated features is: 12
--------------------
feature is: HouseStyle
# of train unique values is: 8
# of test unique values is: 7
   Unique  train_count  train_percent  test_count  test_percent
4  1Story         2089      49.773648         745     51.062371
0  2Story         1261      30.045270         427     29.266621
7  1.5Fin          457      10.888730         160     10.966415
6    SLvl          189       4.503217          63      4.318026
2  SFoyer          118       2.811532          46      3.152844
1  2.5Unf           34       0.810102          13      0.891021


   Unique  train_count  train_percent  test_count  test_percent
4  CBlock         1829      43.578747         601     41.192598
1   PConc         1824      43.459614         661     45.305003
3  BrkTil          447      10.650465         165     11.309116
0    Slab           72       1.715511          25      1.713502
2   Stone           17       0.405051           5      0.342700
5    Wood            8       0.190612           2      0.137080
--------------------
Total number of navigated features is: 22
--------------------
feature is: BsmtQual
# of train unique values is: 6
# of test unique values is: 5
  Unique  train_count  train_percent  test_count  test_percent
3     TA         1886      44.936860         634     43.454421
2     Gd         1749      41.672623         591     40.507197
5     Ex          324       7.719800         137      9.389993
1     Fa          121       2.883012          53      3.632625
0   None          115       2.740052          44      3.015764
4     Po

   Unique  train_count  train_percent  test_count  test_percent
3     WD          2536      60.424112           0      0.000000
5      WD         1267      30.188230        1259     86.291981
4     New          239       5.694544         117      8.019191
7     COD           87       2.072909          44      3.015764
6   ConLD           26       0.619490          17      1.165182
1     CWD           12       0.285919           8      0.548321
10  ConLI            9       0.214439           4      0.274160
0   ConLw            8       0.190612           3      0.205620
8     Oth            7       0.166786           4      0.274160
9     Con            5       0.119133           3      0.205620
2     VWD            1       0.023827           0      0.000000
--------------------
Total number of navigated features is: 42
--------------------
feature is: SaleCondition
# of train unique values is: 6
# of test unique values is: 6
    Unique  train_count  train_percent  test_count  test_perc

In [191]:
for i in df_train.select_dtypes(include= 'object').columns:
    Max = max([len(df_train[df_train[i] == j]) for j in df_train[i].unique()])
    Percent = round(100*Max/len(df_train))
    if Percent >= 90:
        print(i)
    else:
        pass

Street
LandContour
Utilities
LandSlope
Condition2
RoofMatl
BsmtCond
Heating
CentralAir
Electrical
Functional
GarageQual
GarageCond
PavedDrive


## By simple analysis you can conclude that:

MSZoning:-
  - Train: convert 'A (agr)' and 'I (all)' to 'C (all)'
Neighborhood:-
  - Train: convert 'Greens', 'GrnHill' and 'Landmrk' to 'NAmes'
Condition1 and Condition2
  - To be combined in a new feature 'Condition' and put all values that have count less than 4 in a new category 'other'
HouseStyle:-
  - Train: convert category '2.5Fin' to '2.5Unf'
Exterior1st:-
  - Train: convert categories 'Stone', 'PreCast', 'ImStucc' to 'BrkComm'
Exterior2nd:-
  - Train: convert categories 'PreCast', and 'Other' to 'CBlock'
BsmtQual:-
  - Train: convert 'Po' to 'Fa'
KitchenQual:-
  - Train: convert 'Po' to 'Fa'
SaleType:-
  - Train: convert 'VWD' to 'Con'
  - Train: convert 'WD ' to 'WD'
Some featues could be deleted
  - Street, Heating, PoolQC
Some features maybe better to be ordinal encoded
  - ExterQual:- 
                  Po : 1, Fa : 2, TA : 3, Gd : 4, Ex : 5
  - ExterCond:- 
                  Po : 1, Fa : 2, TA : 3, Gd : 4, Ex : 5
  - BsmtQual:- 
                  None : 0, Po : 1, Fa : 2, TA : 3, Gd : 4, Ex 
    : 5
  - BsmtExposure:- 
                  None : 0, No : 1, Mn : 2, Av : 3, Gd : 4
  - BsmtFinType1:- 
                  None : 0, Unf : 1, LwQ : 2, Rec : 3, BLQ : 4, ALQ : 5, GLQ : 6
  - BsmtFinType2:-
                  None : 0, Unf : 1, LwQ : 2, Rec : 3, BLQ : 4, ALQ : 5, GLQ : 6
  - HeatingQC:- 
                  Po : 1, Fa : 2, TA : 3, Gd : 4, Ex : 5
  - KitchenQual:- 
                  Po : 1, Fa : 2, TA : 3, Gd : 4, Ex : 5
  - GarageFinish:- 
                  None : 0, Unf : 1, RFn : 2, Fin : 3
  - GarageQual:- 
                  None : 0, Po : 1, Fa : 2, TA : 3, Gd : 4, Ex 
                  
   : 5
Drop any feature that have unique value more than 90%
Street, LandContour, Utilities, LandSlope, RoofMatl, BsmtCond, Heating, CentralAir, Electrical, Functional, GarageQual, GarageCond, PavedDrive

In [192]:
cols_to_drop_2 = ['Street', 'LandContour', 'Utilities', 'LandSlope', 'RoofMatl', 'BsmtCond',
                  'Heating', 'CentralAir', 'Electrical', 'Functional', 'GarageQual', 
                  'GarageCond', 'PavedDrive']

In [193]:
df_train = df_train.drop(cols_to_drop_2, axis = 1)
df_test = df_test.drop(cols_to_drop_2, axis = 1)

In [194]:
df_train['MSZoning'] = df_train['MSZoning'].replace(to_replace=['A (agr)', 'I (all)'], value = 'C (all)', inplace=False)


In [195]:
df_train['Neighborhood'] = df_train['Neighborhood'].replace(to_replace=['Greens', 'GrnHill','Landmrk'], value = 'NAmes', inplace=False)

In [196]:
df_train['Condition'] = df_train['Condition1'] + '_' + df_train['Condition2']
df_test['Condition'] = df_test['Condition1'] + '_' + df_test['Condition2']

In [197]:
df_train = df_train.drop(['Condition1', 'Condition2'], axis =1)
df_test = df_test.drop(['Condition1', 'Condition2'], axis =1)

In [198]:
condition_unique_vaues_totoal = set(df_train['Condition'].unique()).union(set(df_test['Condition'].unique()))
unique_count_train = len(df_train['Condition'].unique())
unique_count_test = len(df_test['Condition'].unique())
print(f"No. of train unique values is: {unique_count_train}")
print(f"No. of test unique values is: {unique_count_test}")
df_count = pd.DataFrame({'Unique': list(condition_unique_vaues_totoal)})
df_count['train_count'] = [len(df_train[df_train['Condition'] == j]) for j in condition_unique_vaues_totoal]
df_count['train_percent'] = (df_count['train_count']/len(df_train)) * 100
df_count['test_count'] = [len(df_test[df_test['Condition'] == j]) for j in condition_unique_vaues_totoal]
df_count['test_percent'] = (df_count['test_count']/len(df_test)) * 100
df_count.sort_values('train_count', ascending = False)


No. of train unique values is: 22
No. of test unique values is: 16


,Unique,train_count,train_percent,test_count,test_percent
21,Norm_Norm,3611,86.037646,1251,85.743660
8,Feedr_Norm,225,5.360972,79,5.414668
19,Artery_Norm,130,3.097451,44,3.015764
7,RRAn_Norm,58,1.381939,19,1.302262
20,PosN_Norm,51,1.215154,18,1.233722
14,RRAe_Norm,37,0.881582,17,1.165182
17,PosA_Norm,25,0.595664,9,0.616861
18,RRAn_Feedr,12,0.285919,4,0.274160
12,RRNn_Norm,10,0.238265,3,0.205620
16,RRNe_Norm,8,0.190612,4,0.274160


In [199]:
to_replace_train = [i for i in df_train['Condition'].unique() if df_train['Condition'].value_counts()[i] < 5]
to_replace_test = [i for i in df_test['Condition'].unique() if df_test['Condition'].value_counts()[i] < 5]
df_train['Condition'] = df_train['Condition'].replace(to_replace= to_replace_train, value = 'Other', inplace=False)
df_test['Condition'] = df_test['Condition'].replace(to_replace = to_replace_test, value = 'Other', inplace = False)
df_train['Condition'] = df_train['Condition'].replace(to_replace=['RRAn_Feedr', 'RRNn_Norm', 'RRNe_Norm', 'PosN_PosN', 'Feedr_Feedr']
                                                              , value = 'Other', inplace=False)

In [200]:
df_train['HouseStyle'] = df_train['HouseStyle'].replace(to_replace=['2.5Fin'], value = '2.5Unf', inplace=False)

In [201]:
df_train['Exterior1st'] = df_train['Exterior1st'].replace(to_replace=['Stone', 'PreCast', 'ImStucc'], value = 'BrkComm', inplace=False)


In [202]:
df_train['Exterior2nd'] = df_train['Exterior2nd'].replace(to_replace=['PreCast', 'Other'], value = 'CBlock', inplace=False)

In [203]:
df_train['BsmtQual'] = df_train['BsmtQual'].replace(to_replace=['Po'], value = 'Fa', inplace=False)


In [204]:
df_train['KitchenQual'] = df_train['KitchenQual'].replace(to_replace=['Po'], value = 'Fa', inplace=False)

In [205]:
df_train['SaleType'] = df_train['SaleType'].replace(to_replace=['VWD'], value = 'Con', inplace=False)
df_train['SaleType'] = df_train['SaleType'].replace(to_replace=['WD '], value = 'WD', inplace=False)

In [206]:
# Recheck categorical feature to ensure that every thing is okay
compare_cols(cols = df_train.select_dtypes(include= 'object').columns)

feature is: MSSubClass
# of train unique values is: 16
# of test unique values is: 16
   Unique  train_count  train_percent  test_count  test_percent
0      20         1524      36.311651         543     37.217272
3      60          828      19.728377         276     18.917066
7      50          422      10.054801         143      9.801234
6     120          259       6.171075          95      6.511309
15     30          204       4.860615          70      4.797807
12    160          186       4.431737          65      4.455106
9      70          185       4.407910          68      4.660727
10     80          174       4.145818          60      4.112406
5      90          156       3.716941          57      3.906785
14    190           87       2.072909          31      2.124743
11     85           67       1.596378          28      1.919123
13     75           38       0.905409           7      0.479781
1      45           30       0.714796           6      0.411241
4     180         

    Unique  train_count  train_percent  test_count  test_percent
1    Gable         3319      79.080295        1169     80.123372
2      Hip          787      18.751489         265     18.163125
0     Flat           33       0.786276           7      0.479781
4  Gambrel           33       0.786276          11      0.753941
5  Mansard           18       0.428878           4      0.274160
3     Shed            7       0.166786           3      0.205620
--------------------
Total number of navigated features is: 10
--------------------
feature is: Exterior1st
# of train unique values is: 13
# of test unique values is: 13
     Unique  train_count  train_percent  test_count  test_percent
3   VinylSd         1433      34.143436         511     35.023989
1   HdBoard          650      15.487253         220     15.078821
6   MetalSd          647      15.415773         230     15.764222
10  Wd Sdng          614      14.629497         205     14.050720
11  Plywood          322       7.672147     

    Unique  train_count  train_percent  test_count  test_percent
6   2005.0          750      17.869907         288     19.739548
12  2000.0          393       9.363831         127      8.704592
23  1975.0          307       7.314749         103      7.059630
15  1995.0          301       7.171789         110      7.539411
17  1965.0          267       6.361687          87      5.962988
21  1960.0          261       6.218728          90      6.168609
7   1970.0          244       5.813676          75      5.140507
11  1955.0          235       5.599238          79      5.414668
4      0.0          232       5.527758          78      5.346127
16  1980.0          199       4.741482          69      4.729267
13  1990.0          169       4.026686          47      3.221385
3   1950.0          165       3.931380          65      4.455106
20  1985.0          120       2.859185          36      2.467443
18  1940.0          112       2.668573          39      2.673064
5   2010.0          110  

In [207]:
# Applying ordinal encodings
cols_OE = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'HeatingQC'
           , 'KitchenQual', 'GarageFinish']
dic = {'None':0, 'Po': 1, 'Fa': 2, 'TA': 3, 'Gd': 4, 'Ex': 5, 'No': 1, 'Mn': 2, 'Av': 3, 'Unf': 1, 'LwQ': 2, 'Rec': 3, 'BLQ': 4
       , 'ALQ': 5, 'GLQ': 6, 'RFn': 2, 'Fin': 3, 'N': 1, 'P': 2, 'Y': 3, 'MnWw': 1, 'GdWo': 2, 'MnPrv': 3, 'GdPrv': 4,'Other':1}

In [208]:
df_train[cols_OE] = df_train[cols_OE].replace(dic, inplace = False)

df_test[cols_OE] = df_test[cols_OE].replace(dic, inplace = False)

## Apply categorical encoding on the remaining categorical features after ordinal encoding

In [209]:
import category_encoders as ce

cols_CE = df_train.select_dtypes(include= 'object').columns
cols_CE

Index(['MSSubClass', 'MSZoning', 'LotShape', 'LotConfig', 'Neighborhood',
       'BldgType', 'HouseStyle', 'YearBuilt', 'YearRemodAdd', 'RoofStyle',
       'Exterior1st', 'Exterior2nd', 'Foundation', 'GarageType', 'GarageYrBlt',
       'MoSold', 'YrSold', 'SaleType', 'SaleCondition', 'Condition'],
      dtype='object')

In [210]:
cbe = ce.cat_boost.CatBoostEncoder(a = 1)
cbe.fit(df_train[cols_CE], df_train['SalePrice'])
df_train[cols_CE] = cbe.transform(df_train[cols_CE])
df_test[cols_CE] = cbe.transform(df_test[cols_CE])

In [211]:
df_train


,MSSubClass,MSZoning,LotFrontage,LotArea,LotShape,LotConfig,Neighborhood,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,Exterior1st,Exterior2nd,ExterQual,ExterCond,Foundation,BsmtQual,BsmtExposure,BsmtFinType1,BsmtFinType2,TotalBsmtSF,HeatingQC,GrLivArea,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,MoSold,YrSold,SaleType,SaleCondition,SalePrice,Id,Bath_total,BsmtFinSF,Porch,FinSF,Condition
0,12.329425,12.076388,65.000000,8450,11.914794,11.991942,12.169479,12.032098,12.161408,7,5,12.334752,12.220780,11.972589,12.203726,12.203591,4,3,12.252504,4,1,6,1,856.0,5,1710,3,1,4,8,12.151564,12.326019,2,2.0,548.0,12.016631,12.006284,11.993244,12.006215,12.247694,0,3.5,556.0,61,1710,12.030107
1,12.042812,12.076388,80.000000,9600,11.914794,12.018549,12.355473,12.032098,11.985543,6,8,11.950272,11.917906,11.972589,11.861523,11.869456,3,3,11.867427,4,4,5,1,1262.0,5,1262,3,1,3,6,12.151564,11.921497,2,2.0,460.0,11.986325,12.036516,11.993244,12.006215,12.109011,1,2.5,694.0,0,1262,11.818231
2,12.329425,12.076388,68.000000,11250,12.172900,11.991942,12.169479,12.032098,12.161408,7,5,12.296827,12.168079,11.972589,12.203726,12.203591,4,3,12.252504,4,2,6,1,920.0,5,1786,3,1,4,6,12.151564,12.242473,2,2.0,608.0,12.058606,12.006284,11.993244,12.006215,12.317167,2,3.5,52.0,42,1786,12.030107
3,11.922851,12.076388,60.000000,9550,12.172900,12.010626,12.192566,12.032098,12.161408,7,5,11.761764,11.783534,11.972589,11.834868,11.871809,3,3,11.705462,3,1,5,1,756.0,4,1717,3,1,4,7,11.758417,12.242473,1,3.0,642.0,12.016631,12.011531,11.993244,11.747216,11.849398,3,2.0,-324.0,307,1717,12.030107
4,12.329425,12.076388,84.000000,14260,12.172900,12.018549,12.668495,12.032098,12.161408,8,5,12.296827,12.168079,11.972589,12.203726,12.203591,4,3,12.252504,4,3,6,1,1145.0,5,2198,4,1,4,9,12.151564,12.242473,2,3.0,836.0,12.032330,12.006284,11.993244,12.006215,12.429216,4,3.5,165.0,84,2198,12.030107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4192,12.017490,12.076388,37.000000,7937,12.172900,12.235561,11.961841,12.032098,12.001097,6,6,12.085039,12.057041,11.972589,11.951102,11.960955,3,3,11.867427,3,3,6,1,1003.0,3,1003,3,1,3,6,11.758417,11.984293,1,2.0,588.0,11.995382,12.011531,11.993244,12.006215,11.867097,4192,2.0,635.0,0,1003,12.030107
4193,12.042812,12.076388,71.527512,8885,12.172900,11.991942,11.961841,12.032098,11.985543,5,5,12.085039,12.057041,11.972589,11.951102,11.960955,3,3,11.867427,4,3,4,5,864.0,3,902,2,1,3,5,12.151564,11.984293,1,2.0,484.0,12.019532,12.011531,11.993244,12.006215,11.782953,4193,2.0,386.0,0,902,12.030107
4194,11.899945,12.076388,62.000000,10441,11.914794,11.991942,11.961841,12.032098,11.827153,5,5,12.226094,12.092630,11.972589,11.951102,11.871809,3,3,12.252504,4,3,6,1,912.0,3,970,3,1,3,6,11.503845,11.507706,1,0.0,0.0,12.030717,12.011531,11.993244,12.006215,11.790557,4194,1.5,-238.0,32,970,12.030107
4195,12.042812,12.076388,77.000000,10010,11.914794,11.991942,11.961841,12.032098,11.985543,5,5,11.950272,11.917906,11.972589,11.951102,11.960955,3,3,11.867427,4,3,5,2,1389.0,4,1389,2,1,3,6,12.151564,11.921497,2,2.0,418.0,11.956925,12.011531,11.993244,12.006215,12.043554,4195,2.0,999.0,38,1389,12.030107


In [212]:
df_test

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,LotShape,LotConfig,Neighborhood,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,Exterior1st,Exterior2nd,ExterQual,ExterCond,Foundation,BsmtQual,BsmtExposure,BsmtFinType1,BsmtFinType2,TotalBsmtSF,HeatingQC,GrLivArea,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,MoSold,YrSold,SaleType,SaleCondition,Bath_total,BsmtFinSF,Porch,FinSF,Condition
0,1461,12.042812,11.769765,80.0,11622,11.914794,11.991942,11.872327,12.032098,11.985543,5,6,11.878762,11.819196,11.972589,12.203726,12.203591,3,3,11.867427,3,1,3,2,882.0,3,896,2,1,3,5,12.151564,11.866392,1,1.0,730.0,12.019532,11.977006,11.993244,12.006215,1.0,342.0,120,896,11.818231
1,1462,12.042812,12.076388,81.0,14267,12.172900,12.010626,11.872327,12.032098,11.985543,6,6,11.878762,11.819196,12.184188,11.834868,11.836000,3,3,11.867427,3,1,5,1,1329.0,3,1329,3,1,4,6,12.151564,11.866392,1,1.0,312.0,12.019532,11.977006,11.993244,12.006215,1.5,517.0,36,1329,12.030107
2,1463,12.329425,12.076388,74.0,13830,12.172900,11.991942,12.145231,12.032098,12.161408,5,5,12.297921,12.168079,11.972589,12.203726,12.203591,3,3,12.252504,4,1,6,1,928.0,4,1629,3,1,3,6,12.151564,12.238944,3,2.0,482.0,11.995382,11.977006,11.993244,12.006215,2.5,654.0,34,1629,12.030107
3,1464,12.329425,12.076388,78.0,9978,12.172900,11.991942,12.145231,12.032098,12.161408,6,6,12.296827,12.168079,11.972589,12.203726,12.203591,3,3,12.252504,3,1,6,1,926.0,5,1604,3,1,4,7,12.151564,12.242473,3,2.0,470.0,12.019532,11.977006,11.993244,12.006215,2.5,278.0,36,1604,12.030107
4,1465,12.184693,12.076388,43.0,5005,12.172900,11.991942,12.570606,12.081073,11.985543,8,5,12.226094,12.092630,11.972589,11.951102,11.960955,4,3,12.252504,4,1,5,1,1280.0,5,1280,2,1,4,5,12.151564,12.120708,2,2.0,506.0,12.010428,11.977006,11.993244,12.006215,2.0,-754.0,226,1280,12.030107
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,11.792504,11.691904,21.0,1936,11.914794,11.991942,11.467787,11.775043,12.161408,4,7,11.825316,11.783534,11.972589,12.155591,12.152845,3,3,11.867427,3,1,1,1,546.0,4,1092,3,1,3,5,11.503845,11.507706,1,0.0,0.0,12.019532,12.011531,11.993244,12.006215,1.5,-546.0,0,1092,12.030107
1455,2916,11.792504,11.691904,21.0,1894,11.914794,11.991942,11.467787,12.081073,12.161408,4,5,11.825316,11.783534,11.972589,12.155591,12.152845,3,3,11.867427,3,1,3,1,546.0,3,1092,3,1,3,6,11.577695,11.894298,1,1.0,286.0,11.956925,12.011531,11.993244,11.747216,1.5,-42.0,24,1092,12.030107
1456,2917,12.042812,12.076388,160.0,20000,11.914794,11.991942,11.961841,12.032098,11.985543,5,7,11.878762,12.166791,11.972589,12.203726,12.203591,3,3,11.867427,3,1,5,1,1224.0,5,1224,4,1,3,7,11.758417,11.866392,1,2.0,576.0,12.058606,12.011531,11.993244,11.747216,2.0,1224.0,0,1224,12.030107
1457,2918,11.899945,12.076388,62.0,10441,11.914794,11.991942,11.961841,12.032098,11.827153,5,5,12.226094,12.092630,11.972589,11.951102,11.871809,3,3,12.252504,4,3,6,1,912.0,3,970,3,1,3,6,11.503845,11.507706,1,0.0,0.0,12.030717,12.011531,11.993244,12.006215,1.5,-238.0,32,970,12.030107


# df_train is a small dataset, so we will train our model on the whole set

In [213]:
from xgboost import XGBRegressor

In [214]:
model_xgb = XGBRegressor(
                        n_estimators = 1000,
                        max_depth = 8,
                        subsample = 0.6,
                        #min_child_weight = 60, 
                        learning_rate = 0.1)

In [215]:
model_xgb.fit(df_train.drop(['Id', 'SalePrice'], axis =1), df_train['SalePrice'],
            verbose = False)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, feature_types=None, gamma=0, gpu_id=-1,
             grow_policy='depthwise', importance_type=None,
             interaction_constraints='', learning_rate=0.1, max_bin=256,
             max_cat_threshold=64, max_cat_to_onehot=4, max_delta_step=0,
             max_depth=8, max_leaves=0, min_child_weight=1, missing=nan,
             monotone_constraints='()', n_estimators=1000, n_jobs=0,
             num_parallel_tree=1, predictor='auto', random_state=0, ...)

In [216]:
y_test = model_xgb.predict(df_test.drop('Id', axis = 1))
y_test

array([11.562249, 12.055898, 12.153097, ..., 11.782358, 11.789934,
       12.145558], dtype=float32)

In [217]:
sub = pd.DataFrame({'Id': range(1461,2920), 'SalePrice': np.exp(y_test)})
sub = sub.set_index('Id')
sub.to_csv('House.csv')